### Two-Opt Algorithm and SA Algorithm (Seperated)


In [ ]:
# Use two opt
def two_opt(routes, adj_matrix, num_iterations):
    """
    Enhanced Two-Opt heuristic for route optimization with time constraints.
    """
    best_routes = routes.copy()
    best_distance = sum(calculate_total_distance(route, adj_matrix) for route in best_routes)

    for _ in range(num_iterations):
        # Select a random route
        selected_route_index = np.random.randint(0, len(routes))
        selected_route = routes[selected_route_index]

        # Randomly pick two indices
        i, j = np.random.randint(1, len(selected_route) - 1, size=2)
        if j < i:
            i, j = j, i

        # Create a new route by reversing segment i to j
        new_route = selected_route.copy()
        new_route[i:j] = selected_route[j - 1:i - 1:-1]

        # Validate the route with time windows
        new_routes = routes.copy()
        new_routes[selected_route_index] = new_route

            # Calculate the cost for the new solution
        new_cost = sum(calculate_total_distance(route, adj_matrix) for route in new_routes)
        if new_cost < best_distance:
            best_routes = new_routes
            best_distance = new_cost

    return best_routes

def simulated_annealing(routes, adj_matrix, num_iterations, temperature):
    best_routes = routes.copy()
    current_routes = routes.copy()
    best_distance = sum(calculate_total_distance(route, adj_matrix) for route in best_routes)

    for _ in range(num_iterations):
        temp = max(0.01, temperature * (1 - (_ / num_iterations)))  # Cool down

        # Random perturbation
        selected_route_index = np.random.randint(0, len(current_routes))
        i, j = np.random.randint(1, len(current_routes[selected_route_index]) - 1, size=2)
        if j < i: i, j = j, i
        perturbed_route = current_routes[selected_route_index].copy()
        perturbed_route[i:j] = current_routes[selected_route_index][j - 1:i - 1:-1]

        # Cost calculation
        perturbed_routes = current_routes.copy()
        perturbed_routes[selected_route_index] = perturbed_route
        perturbed_cost = sum(calculate_total_distance(route, adj_matrix) for route in perturbed_routes)

        # Accept or reject
        if (perturbed_cost < best_distance or
            np.random.rand() < np.exp((best_distance - perturbed_cost) / temp)):
            current_routes = perturbed_routes
            if perturbed_cost < best_distance:
                best_routes = perturbed_routes
                best_distance = perturbed_cost

    return best_routes



def vrp_solver2(location, num_iterations):
    """
    Solve the VRP using the provided location for coordinates, vehicle capacity,
    and number of iterations for the two-opt optimization.
    """
    nb_vehicle, capacity_max, coordinates, demand, earliest_time, latest_time, service_time = read_txt_file(location)
    adj_matrix = adjacency_matrix(coordinates)
    all_routes, capacity_list, route_times, start_list = nearest_neighbor_with_time_windows(adj_matrix, demand, int(capacity_max), earliest_time, latest_time, service_time)
    optimized_routes = two_opt(all_routes, adj_matrix, num_iterations)
    #optimized_routes = simulated_annealing(all_routes, adj_matrix, num_iterations, 2000)
    route_distances = [calculate_total_distance(route, adj_matrix) for route in optimized_routes]
    formatted_routes = format_output(optimized_routes)
    return formatted_routes, route_distances, capacity_list, route_times, coordinates, start_list

# Main execution
num_iterations = 100000
solution, route_distances, capacity_list, route_times, coordinates, start_list = vrp_solver2(location, num_iterations)

# Display results
n = 0
print("Total cost: " + str(ceil(cost)))
for route, distance, route_time, start in zip(solution, route_distances, route_times, start_list):
    end = convert_time(start + route_time)
    print(f"Route {n+1}: {route}\n"
          f"Truck capacity: {capacity_list[n]}\n"
          f"Total distance travelled (cost): {ceil(distance)}\n"
          f"Starting time: {convert_time(start)}\n"
          f"Ending time: {end}\n"          
          f"Total time: {convert_time(route_time)}\n"
          )
    plot_route_with_distances(coordinates, route, f"{dataset_name.upper()} - Route {n+1}", 5)
    n += 1
    
plot_global_routes(coordinates, solution, dataset_name, 10)


NameError: name 'cost' is not defined

### Tabu list

In [ ]:
# --- Correction de la fonction de nearest_neighbor_with_time_windows ---
def nearest_neighbor_with_time_windows(adj_matrix, demands, capacity_max, earliest_time, latest_time, service_time):
    """
    Nearest Neighbor heuristic with time windows.
    """
    num_points = adj_matrix.shape[0]  # Count rows in matrix to get the number of points
    visited = [False] * num_points  # Boolean array to inform if a point was visited or not
    routes = []  # Array to store multiple routes
    capacity_list = []  # Array storing the final capacity of each route
    time_list = []  # Array keeping total time needed for each route in mind
    start_list = []  # Array storing starting time of one travel

    while sum(visited) < num_points:  # While all points are not visited
        current_node = 0  # Start at the depot
        current_capacity = 0  # Initialize empty capacity
        current_time = earliest_time[current_node]  # Initialize current_time to the READY TIME of the depot (often 0)
        route = [current_node]  # Array storing the current route starting with the depot
        visited[current_node] = True  # Mark depot as visited

        while True:
            last_node = route[-1]  # last_node is the last element in route
            nearest = None  # nearest corresponds to the next nearest neighbor (which is unknown here)
            min_distance = float('inf')  # Initialize min_distance to infinity

            for neighbor in [i for i, v in enumerate(visited) if not v]:
                travel_time = adj_matrix[last_node, neighbor]  # Time = distance (simplification)
                arrival_time = current_time + travel_time  # Calculate the arrival time

                if (demands[neighbor] + current_capacity <= capacity_max  # Capacity is not exceeded
                        and arrival_time <= latest_time[neighbor]  # Arrival within the latest time window
                        and arrival_time >= earliest_time[neighbor]  # Arrival after the earliest time
                        and adj_matrix[last_node, neighbor] < min_distance):  # Find nearest neighbor
                    nearest = neighbor
                    min_distance = adj_matrix[last_node, neighbor]

            if nearest is None:  # No more valid neighbors, end this route
                break

            # Update route, time, and capacity
            travel_time = adj_matrix[last_node, nearest]
            current_time += travel_time
            current_time = max(current_time, earliest_time[nearest])  # Wait if arriving early
            current_time += service_time[nearest]  # Add service time
            route.append(nearest)
            visited[nearest] = True
            current_capacity += demands[nearest]

        # Return to depot
        travel_time = adj_matrix[route[-1], 0]  # Travel time between last node and the depot
        current_time += travel_time
        route.append(0)  # End the route at the depot

        routes.append(route)  # Add the route to all routes
        capacity_list.append(current_capacity)  # Add the current capacity to the capacity list
        time_list.append(current_time)  # Add the current time to the time list
        start_list.append(max(0, earliest_time[route[1]] - adj_matrix[route[0], route[1]]))  # Calculate start time

    return routes, capacity_list, time_list, start_list


# --- Fonction d'évaluation ---
def evaluate_solution(routes, adj_matrix):
    """
    Calculate the total distance cost of a given solution (sum of the distances of all routes).
    """
    total_dist = 0
    for route in routes:
        total_dist += calculate_total_distance(route, adj_matrix)
    return total_dist


# --- Tabu Search ---
def swap_customers(route1, route2):
    """
    Swap two customers between two routes.
    """
    i = random.randint(1, len(route1) - 1)  # Exclude depot (index 0)
    j = random.randint(1, len(route2) - 1)  # Exclude depot (index 0)
    route1_new = route1[:]
    route2_new = route2[:]
    route1_new[i], route2_new[j] = route2_new[j], route1_new[i]
    return route1_new, route2_new


def tabu_search(initial_routes, adj_matrix, max_iter=100, tabu_tenure=10):
    """
    Perform Tabu Search to improve the initial solution.
    """
    current_solution = initial_routes
    best_solution = current_solution
    best_distance = evaluate_solution(current_solution, adj_matrix)
    tabu_list = []  # Tabu list to store recent changes

    # Perform Tabu Search for a fixed number of iterations
    for iteration in range(max_iter):
        neighbors = []
        for i in range(len(current_solution)):
            for j in range(i + 1, len(current_solution)):
                route1, route2 = swap_customers(current_solution[i], current_solution[j])
                neighbors.append((route1, route2))

        # Evaluate neighbors
        best_neighbor = None
        best_neighbor_cost = float('inf')

        for neighbor in neighbors:
            if neighbor not in tabu_list:  # Check if neighbor is not in tabu list
                neighbor_cost = evaluate_solution(neighbor, adj_matrix)
                if neighbor_cost < best_neighbor_cost:
                    best_neighbor = neighbor
                    best_neighbor_cost = neighbor_cost

        # Update the solution
        if best_neighbor_cost < best_distance:
            best_solution = best_neighbor
            best_distance = best_neighbor_cost

        # Update Tabu List
        tabu_list.append(best_neighbor)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Remove the oldest tabu entry

        print(f"Iteration {iteration + 1}/{max_iter}, Best Cost: {best_distance}")

    return best_solution, best_distance


# --- Main Solver with Tabu Search ---
def vrp_solver_with_tabu(location, max_iter=100, tabu_tenure=10):
    nb_vehicle, capacity_max, coordinates, demand, earliest_time, latest_time, service_time = read_txt_file(location)
    adj_matrix = adjacency_matrix(coordinates)
    initial_routes, capacity_list, route_distances, route_times, start_list = nearest_neighbor_with_time_windows(
        adj_matrix, demand, int(capacity_max), earliest_time, latest_time, service_time
    )

    # Améliorer la solution en utilisant Tabu Search
    optimized_routes, optimized_cost = tabu_search(initial_routes, adj_matrix, max_iter, tabu_tenure)

    # Afficher la solution optimisée
    print("Optimized Total Cost: ", ceil(optimized_cost))
    return optimized_routes, optimized_cost

optimized_solution, optimized_cost = vrp_solver_with_tabu(location)


KeyboardInterrupt: 